In [33]:
from flask import Flask, request, jsonify, render_template
import random
from tensorflow.keras.models import load_model
import numpy as np
import librosa
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [34]:
app = Flask(__name__, template_folder='template', static_folder='static')

model=load_model(r'C:\Users\This PC\Downloads\SER\model.h5')

In [35]:
@app.route('/')
def home():
    return render_template('index.html')

In [36]:
@app.route('/predict_',methods=['POST'])
def predict_():

    # get file from POST request and save it
    audio_file = request.files["file"]
    file_name = str(random.randint(0, 100000))
    audio_file.save(file_name)

    Features = pd.read_csv(r'C:\Users\This PC\Downloads\SER\features.csv')
    Y = Features['labels'].values
    encoder = OneHotEncoder()
    Y = encoder.fit_transform(np.array(Y).reshape(-1,1)).toarray()

    X, sample_rate = librosa.load(file_name, res_type='kaiser_fast',duration=2.5,sr=22050*2,offset=0.5)
    sample_rate = np.array(sample_rate)
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13),axis=0)
    featurelive = mfccs
    livedf2 = featurelive
    livedf2= pd.DataFrame(data=livedf2)
    livedf2 = livedf2.stack().to_frame().T
    twodim= np.expand_dims(livedf2, axis=2)



    prediction = model.predict(twodim)
    livepredictions = (encoder.inverse_transform((prediction)))

    return render_template('index.html', prediction_text='OUTPUT IS {}'.format(livepredictions))


In [37]:
@app.route('/results',methods=['POST'])
def results():
    data = request.get_json(force=True)
    prediction = model.predict(data.values())
    
    return jsonify(prediction)

In [38]:
if __name__ == "__main__":
    app.run(debug=True,use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Jul/2021 17:57:00] "GET / HTTP/1.1" 200 -


127.0.0.1 - - [30/Jul/2021 17:57:16] "POST /predict_ HTTP/1.1" 200 -
127.0.0.1 - - [30/Jul/2021 17:58:15] "POST /predict_ HTTP/1.1" 200 -
C:\Users\This PC\anaconda3\lib\site-packages\librosa\core\audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
127.0.0.1 - - [30/Jul/2021 17:58:19] "POST /predict_ HTTP/1.1" 500 -
Traceback (most recent call last):
  File "C:\Users\This PC\anaconda3\Lib\site-packages\librosa\core\audio.py", line 149, in load
    with sf.SoundFile(path) as sf_desc:
  File "C:\Users\This PC\anaconda3\Lib\site-packages\soundfile.py", line 629, in __init__
    self._file = self._open(file, mode_int, closefd)
  File "C:\Users\This PC\anaconda3\Lib\site-packages\soundfile.py", line 1183, in _open
    _error_check(_snd.sf_error(file_ptr),
  File "C:\Users\This PC\anaconda3\Lib\site-packages\soundfile.py", line 1357, in _error_check
    raise RuntimeError(prefix + _ffi.string(err_str).decode